In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
import os
from math import sqrt, atan, asin, acos, sin, cos, radians
pd.options.display.max_columns = None

In [2]:
def open_leitura(input_path):
    # Obtém a extensão do arquivo
    _, file_extension = os.path.splitext(input_path)

    # Verifica a extensão do arquivo
    if file_extension.lower() == '.csv':
        tipo_arquivo = 'csv'
    elif file_extension.lower() == '.xlsx':
        tipo_arquivo = 'excel'
    elif file_extension.lower() == '.xls':
        tipo_arquivo = 'excel'


    if tipo_arquivo=='excel':
        df_leitura=pd.read_excel(input_path) #Leitura interna da planilha de dados primária
    elif tipo_arquivo=='csv':
        df_leitura = pd.read_csv(input_path)
    return df_leitura
    
def open_m_conv(conv_path, grav):
    df_conv=pd.read_excel(conv_path, sheet_name=str(grav)) #Leitura interna da planilha de conversão
    return df_conv

In [3]:
#-->TRANSLATE AND REFACTORATE
def calculate_julian_century(timestamp):
    """
    Take a datetime object and returns the decimal Julian century and
    floating point hour. This is in reference to noon on December 31,
    1899 as stated in the paper.
    """
    origin_date = datetime(1899, 12, 31, 12, 00, 00)  # Noon Dec 31, 1899
    dt = timestamp - origin_date
    days = dt.days + dt.seconds/3600./24.
    return days/36525, timestamp.hour + timestamp.minute/60. + timestamp.second/3600.

def solve_longman(lat, lon, alt, time):
    """
    Given the location and datetime object, computes the current
    gravitational tide and associated quantities. Latitude and longitude
    and in the traditional decimal notation, altitude is in meters, time
    is a datetime object.
    """

    T, t0 = calculate_julian_century(time)

    if t0 < 0:
        t0 += 24.
    if t0 >= 24:
        t0 -= 24.

    mu = 6.673e-8  # Newton's gravitational constant
    M = 7.3537e25  # Mass of the moon in grams
    S = 1.993e33  # Mass of the sun in grams
    e = 0.05490  # Eccentricity of the moon's orbit
    m = 0.074804  # Ratio of mean motion of the sun to that of the moon
    c = 3.84402e10  # Mean distance between the centers of the earth and the moon
    c1 = 1.495e13  # Mean distance between centers of the earth and sun in cm
    h2 = 0.612  # Love parameter
    k2 = 0.303  # Love parameter
    a = 6.378270e8  # Earth's equitorial radius in cm
    i = 0.08979719  # (i) Inclination of the moon's orbit to the ecliptic
    omega = radians(23.452)  # Inclination of the Earth's equator to the ecliptic 23.452 degrees
    L = -1 * lon  # For some reason his lat/lon is defined with W as + and E as -
    lamb = radians(lat)  # (lambda) Latitude of point P
    H = alt * 100.  # (H) Altitude above sea-level of point P in cm

    # Lunar Calculations
    # (s) Mean longitude of moon in its orbit reckoned from the referred equinox
    s = 4.72000889397 + 8399.70927456 * T + 3.45575191895e-05 * T * T + 3.49065850399e-08 * T * T * T
    # (p) Mean longitude of lunar perigee
    p = 5.83515162814 + 71.0180412089 * T + 0.000180108282532 * T * T + 1.74532925199e-07 * T * T * T
    # (h) Mean longitude of the sun
    h = 4.88162798259 + 628.331950894 * T + 5.23598775598e-06 * T * T
    # (N) Longitude of the moon's ascending node in its orbit reckoned from the referred equinox
    N = 4.52360161181 - 33.757146295 * T + 3.6264063347e-05 * T * T +  3.39369576777e-08 * T * T * T
    # (I) Inclination of the moon's orbit to the equator
    I = acos(cos(omega)*cos(i) - sin(omega)*sin(i)*cos(N))
    # (nu) Longitude in the celestial equator of its intersection A with the moon's orbit
    nu = asin(sin(i)*sin(N)/sin(I))
    # (t) Hour angle of mean sun measured west-ward from the place of observations
    t = radians(15. * (t0 - 12) - L)

    # (chi) right ascension of meridian of place of observations reckoned from A
    chi = t + h - nu
    # cos(alpha) where alpha is defined in eq. 15 and 16
    cos_alpha = cos(N)*cos(nu)+sin(N)*sin(nu)*cos(omega)
    # sin(alpha) where alpha is defined in eq. 15 and 16
    sin_alpha = sin(omega)*sin(N)/sin(I)
    # (alpha) alpha is defined in eq. 15 and 16
    alpha = 2*atan(sin_alpha/(1+cos_alpha))
    # (xi) Longitude in the moon's orbit of its ascending intersection with the celestial equator #1
    xi = N-alpha

    # (sigma) Mean longitude of moon in radians in its orbit reckoned from A
    sigma = s - xi
    # (l) Longitude of moon in its orbit reckoned from its ascending intersection with the equator
    l = sigma + 2*e*sin(s-p)+(5./4)*e*e*sin(2*(s-p)) + (15./4)*m*e*sin(s-2*h+p) + (11./8)*m*m*sin(2*(s-h))

    # Sun
    # (p1) Mean longitude of solar perigee
    p1 = 4.90822941839 + 0.0300025492114 * T +  7.85398163397e-06 * T * T + 5.3329504922e-08 * T * T * T
    # (e1) Eccentricity of the Earth's orbit
    e1 = 0.01675104-0.00004180*T - 0.000000126*T*T
    # (chi1) right ascension of meridian of place of observations reckoned from the vernal equinox
    chi1 = t+h
    # (l1) Longitude of sun in the ecliptic reckoned from the vernal equinox
    l1 = h + 2*e1*sin(h-p1)
    # cosine(theta) Theta represents the zenith angle of the moon
    cos_theta = sin(lamb)*sin(I)*sin(l) + cos(lamb)*(cos(0.5*I)**2 * cos(l-chi) + sin(0.5*I)**2 * cos(l+chi))
    # cosine(phi) Phi represents the zenith angle of the run
    cos_phi = sin(lamb)*sin(omega)*sin(l1) + cos(lamb)*(cos(0.5*omega)**2 * cos(l1-chi1)+sin(0.5*omega)**2*cos(l1+chi1))

    # Distance
    # (C) Distance parameter, equation 34
    C = sqrt(1./(1+0.006738*sin(lamb)**2))
    # (r) Distance from point P to the center of the Earth
    r = C*a + H
    # (a') Distance parameter, equation 31
    aprime = 1./(c*(1-e*e))
    # (a1') Distance parameter, equation 31
    aprime1 = 1./(c1*(1-e1*e1))
    # (d) Distance between centers of the Earth and the moon
    d = 1./((1./c) + aprime*e*cos(s-p)+aprime*e*e*cos(2*(s-p)) + (15./8)*aprime*m*e*cos(s-2*h+p) + aprime*m*m*cos(2*(s-h)))
    # (D) Distance between centers of the Earth and the sun
    D = 1./((1./c1) + aprime1*e1*cos(h-p1))

    # (gm) Vertical componet of tidal acceleration due to the moon
    gm = (mu*M*r/(d*d*d))*(3*cos_theta**2-1) + (3./2)*(mu*M*r*r/(d*d*d*d))*(5*cos_theta**3 - 3*cos_theta)
    # (gs) Vertical componet of tidal acceleration due to the sun
    gs = mu*S*r/(D*D*D) * (3*cos_phi**2-1)

    love = (1+h2-1.5*k2)
    g0 = (gm+gs)*1e3*love
    return g0

In [4]:
def calcular_val_dec(row, g, m, s):
    if row[g] >= 0:
        return row[g] + (row[m] / 60) + (row[s] / 3600)
    else:
        return row[g] - (row[m] / 60) - (row[m] / 3600)
    
def ajustar_hora_utc(hora_utc):
    if hora_utc < 0:
        return hora_utc + 24
    elif hora_utc >= 24:
        return hora_utc - 24
    else:
        return hora_utc

def calcular_dados(df_leitura, fuso_horario):
    #Cálculo de Latitude em Graus decimais
    df_leitura['lat_g_dec'] = df_leitura.apply(calcular_val_dec, axis=1, g='lat_gra', m='lat_min', s='lat_seg' )
    df_leitura['lat_rad'] = np.radians(df_leitura['lat_g_dec'])
    
    #Cálculo de Longitude em Graus decimais
    df_leitura['lon_g_dec'] = df_leitura.apply(calcular_val_dec, axis=1, g='lon_gra', m='lon_min', s='lon_seg' )
    df_leitura['lon_rad'] = np.radians(df_leitura['lon_g_dec'])
    
    #Cálculo do tempo em Horas decimais
    df_leitura['hora_dec'] = (df_leitura['hora']) + (df_leitura['minuto']/(60))

    #Horas (sem minutos e segundos) em UTC
    df_leitura['hora_utc'] = (df_leitura['hora_dec'] - fuso_horario)
    df_leitura['hora_utc'] = df_leitura['hora_utc'].apply(ajustar_hora_utc)

    return df_leitura

In [5]:
#Correções e Transformações importantes
#--------------------------------------------------
def media_leituras(df_leitura):
    #Média das 3 leituras
    df_leitura['g_med_lido'] = (df_leitura['g_l1'] + df_leitura['g_l2'] + df_leitura['g_l3'])/3 
    return df_leitura

def conv_linha(value, df_conv):
    result = df_conv[(value - df_conv['gc1'] >=0) & (value - df_conv['gc1'] < 100)]
    corr = result['gf1'] + (value - result['gc1']) * result['gf0'] 
    return float(corr)

def conversão_mgal(df_leitura, df_conv):
    #Conversão instrumental para mGal
    df_leitura['g_conv'] = df_leitura['g_med_lido'].apply(lambda x: conv_linha(x, df_conv))
    return df_leitura

def correcao_altura_instrumental(df_leitura):
    #Correção de Altura Instrumental
    df_leitura['c_ai'] = 0.308596 * df_leitura['h_instrumento']
    df_leitura['g_ai'] = df_leitura['g_conv'] + df_leitura['c_ai']
    return df_leitura

def correcao_mare(df_leitura, ano, mes, dia):
    df_leitura['t_datetime'] = pd.to_datetime(f"{ano}-{mes}-{dia} ") + pd.to_timedelta(df_leitura['hora_utc'], unit='h')
    #Correção de maré
    df_leitura['c_tide'] = df_leitura.apply(lambda row: solve_longman(row['lat_g_dec'], row['lon_g_dec'], row['alt_m'], row['t_datetime']), axis=1)
    df_leitura['g_tide'] = df_leitura['g_ai'] + df_leitura['c_tide']
    return df_leitura

def correcao_deriva_instrumental(df_leitura):
    delta_g = float(df_leitura['g_tide'].iloc[-1] - df_leitura['g_tide'].iloc[0])
    df_leitura['time_delta'] = ((df_leitura['t_datetime'] - df_leitura['t_datetime'].iloc[0])).dt.total_seconds() /3600
    df_leitura['cd'] =  (df_leitura['time_delta'] / df_leitura['time_delta'].iloc[-1]) * delta_g 
    df_leitura['g_cd'] = df_leitura['g_tide'] - df_leitura['cd']
    return df_leitura

def calcular_aceleracao_observada(g_ref, df_leitura,):
    #Cálculo de Aceleração lida absoluta
    df_leitura['g_obs'] = g_ref + (df_leitura['g_cd'] - df_leitura['g_cd'].iloc[0])
    return df_leitura


def calcular_aceleracoes_teoricas(elipsoide, df_leitura):
    #Acelerações teóricas
    if elipsoide=='grs67':
        #Cálculo de Aceleração do GRS67
        df_leitura['g_teor'] = 978031.8*(1+0.0053024*((np.sin(df_leitura['lat_rad']))**2)-0.0000059*((np.sin(2*df_leitura['lat_rad']))**2))

    elif elipsoide=='grs80':
        #Cálculo de Aceleração do GRS80
        df_leitura['g_teor'] = 978032.7*(1+0.0053024*((np.sin(df_leitura['lat_rad']))**2)-0.0000058*((np.sin(2*df_leitura['lat_rad'])**2)))
                
    elif elipsoide=='grs84':
        #Cálculo de Aceleração do GRS84
        df_leitura['g_teor'] = (9.7803267714*((1+0.00193185138639*((np.sin(df_leitura['lat_rad']))**2))/((1-0.00669437999013*((np.sin(df_leitura['lat_rad']))**2)**(0.5)))))*(100000)
    
    #df_leitura['g_teor'] = df_leitura['g_teor']/100 #VERIFY IF NEEDS TO DIVIDE BY 100!!!!!
    return df_leitura

def correcao_ar_livre(df_leitura):
    #Correção Ar-livre
    df_leitura['ca'] = 0.308596*df_leitura['alt_m']
    df_leitura['g_ca'] = df_leitura['g_obs'] + df_leitura['ca'] - df_leitura['g_teor']
    return df_leitura

def correcao_bouguer_simples(df_leitura, densidade):
    #Correção Bouguer Simples
    df_leitura['cb'] = 0.04192 * densidade * df_leitura['alt_m']
    df_leitura['g_cb'] = df_leitura['g_obs'] + df_leitura['ca'] - df_leitura['cb'] - df_leitura['g_teor']
    return df_leitura  


In [6]:
def df_wrapper(df,  dec=3):
    df_final = pd.DataFrame({
        'Ponto': df['ponto'],
        'Leitura média Gravímetro': np.around(df['g_med_lido'], decimals=dec),
        'Leitura média mGal': np.around(df['g_conv'], decimals=dec),
        'Correção de Maré': np.around(df['c_tide'], decimals=dec),
        'leit. Corr. Maré': np.around(df['g_tide'], decimals=dec),
        'Corr. Deriva': np.around(df['cd'], decimals=dec),
        'leit. corr. Deriva': np.around(df['g_cd'], decimals=dec),
        'g. Obs.': np.around(df['g_obs'], decimals=dec),
        'g. Teórico': np.around(df['g_teor'], decimals=dec),
        'Corr. Ar-livre': np.around(df['ca'], decimals=dec),
        'Anom. Ar-livre': np.around(df['g_ca'], decimals=dec),
        'Corr. Bouguer': np.around(df['cb'], decimals=dec),
        'Anom. Bouguer': np.around(df['g_cb'], decimals=dec)
    })
    return df_final


In [7]:
input_path = r'data\GRARED_P_exemplo.csv'
conv_path = r'data\Tabelas_conv_todas.xlsx'
output_path = r'data\output_exemplo.csv'
aceler_abs = 9815.55
fuso = -3
grav = 996
dia = 30
mes = 4
ano = 2023
densidade_b = 3.22
elipsoide = 'grs84'

def pipeline(grav, dia, mes, ano, fuso, densidade_b, aceler_abs, elipsoide, input_path, conv_path, output_path):
    df_conv = open_m_conv(conv_path, grav)
    df = open_leitura(input_path)
    df_1 = calcular_dados(df,fuso)
    df_2 = media_leituras(df_1)
    df_3 = conversão_mgal(df_2, df_conv)
    df_4 = correcao_altura_instrumental(df_3)
    df_5 = correcao_mare(df_4, ano, mes, dia)
    df_6 = correcao_deriva_instrumental(df_5)
    df_7 = calcular_aceleracao_observada(aceler_abs, df_6)
    df_8 = calcular_aceleracoes_teoricas(elipsoide, df_7)
    df_9 = correcao_ar_livre(df_8)
    df_10 = correcao_bouguer_simples(df_9, densidade_b)
    df_final = df_wrapper(df_10)
    return df_final
df = pipeline(grav, dia, mes, ano, fuso, densidade_b, aceler_abs, elipsoide, input_path, conv_path, output_path)
df


C:\Users\Usuario\AppData\Local\Temp\ipykernel_6624\2278282539.py:11: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  return float(corr)


,Ponto,Leitura média Gravímetro,Leitura média mGal,Correção de Maré,leit. Corr. Maré,Corr. Deriva,leit. corr. Deriva,g. Obs.,g. Teórico,Corr. Ar-livre,Anom. Ar-livre,Corr. Bouguer,Anom. Bouguer
0,1,2381.740,2394.133,0.092,2394.228,-0.000,2394.228,9815.550,980873.318,7.715,-971050.054,3.375,-971053.428
1,2,2194.867,2206.217,0.000,2206.220,-0.006,2206.227,9627.548,980821.646,148.126,-971045.971,64.792,-971110.763
2,3,2015.100,2025.462,-0.026,2025.439,-0.008,2025.447,9446.769,980809.937,385.745,-970977.423,168.728,-971146.151
3,4,1922.320,1932.190,-0.037,1932.156,-0.008,1932.164,9353.485,980806.103,516.898,-970935.720,226.096,-971161.815
4,5,1761.400,1770.421,-0.053,1770.370,-0.010,1770.380,9191.702,980805.611,757.603,-970856.306,331.382,-971187.688
5,6,1922.220,1932.089,-0.063,1932.029,-0.011,1932.040,9353.361,980806.103,516.898,-970935.844,226.096,-971161.939
6,7,2015.220,2025.582,-0.063,2025.522,-0.012,2025.534,9446.856,980809.937,385.745,-970977.336,168.728,-971146.064
7,8,2194.950,2206.301,-0.055,2206.249,-0.013,2206.262,9627.584,980821.646,148.126,-971045.936,64.792,-971110.727
8,1,2381.770,2394.164,0.043,2394.209,-0.019,2394.228,9815.550,980873.318,7.715,-971050.054,3.375,-971053.428


In [8]:
df_conv = open_m_conv(conv_path, grav)
df = open_leitura(input_path)
df_1 = calcular_dados(df,fuso)
df_2 = media_leituras(df_1)
df_3 = conversão_mgal(df_2, df_conv)
df_4 = correcao_altura_instrumental(df_3)
df_5 = correcao_mare(df_4, ano, mes, dia)
df_6 = correcao_deriva_instrumental(df_5)
df_7 = calcular_aceleracao_observada(aceler_abs, df_6)
df_8 = calcular_aceleracoes_teoricas(elipsoide, df_7)
df_9 = correcao_ar_livre(df_8)
df_10 = correcao_bouguer_simples(df_9, densidade_b)
df_10

C:\Users\Usuario\AppData\Local\Temp\ipykernel_6624\2278282539.py:11: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  return float(corr)


,ponto,g_l1,g_l2,g_l3,hora,minuto,h_instrumento,lat_gra,lat_min,lat_seg,lon_gra,lon_min,lon_seg,alt_m,lat_g_dec,lat_rad,lon_g_dec,lon_rad,hora_dec,hora_utc,g_med_lido,g_conv,c_ai,g_ai,t_datetime,c_tide,g_tide,time_delta,cd,g_cd,g_obs,g_teor,ca,g_ca,cb,g_cb
0,1,2381.74,2381.74,2381.74,9,30,0.010,-22.8956,0,0,-43.2233,0,0,25,-22.8956,-0.399604,-43.2233,-0.754389,9.500000,12.500000,2381.740000,2394.133466,0.003086,2394.136552,2023-04-30 12:30:00.000000000,0.091855,2394.228406,0.000000,-0.000000,2394.228406,9815.550000,980873.318477,7.71490,-971050.053577,3.374560,-971053.428137
1,2,2194.87,2194.86,2194.87,12,46,0.009,-22.4967,0,0,-44.6783,0,0,480,-22.4967,-0.392641,-44.6783,-0.779783,12.766667,15.766667,2194.866667,2206.217485,0.002777,2206.220262,2023-04-30 15:46:00.000000001,0.000073,2206.220335,3.266667,-0.006256,2206.226592,9627.548185,980821.645707,148.12608,-971045.971442,64.791552,-971110.762994
2,3,2015.10,2015.10,2015.10,13,30,0.012,-22.4063,0,0,-44.7524,0,0,1250,-22.4063,-0.391064,-44.7524,-0.781077,13.500000,16.500000,2015.100000,2025.461842,0.003703,2025.465545,2023-04-30 16:30:00.000000000,-0.026070,2025.439475,4.000000,-0.007661,2025.447136,9446.768730,980809.937020,385.74500,-970977.423290,168.728000,-971146.151290
3,4,1922.32,1922.32,1922.32,13,52,0.011,-22.3767,0,0,-44.7603,0,0,1675,-22.3767,-0.390547,-44.7603,-0.781215,13.866667,16.866667,1922.320000,1932.189635,0.003395,1932.193030,2023-04-30 16:52:00.000000001,-0.037489,1932.155541,4.366667,-0.008363,1932.163904,9353.485498,980806.103346,516.89830,-970935.719548,226.095520,-971161.815068
4,5,1761.40,1761.40,1761.40,14,30,0.010,-22.3729,0,0,-44.7039,0,0,2455,-22.3729,-0.390481,-44.7039,-0.780230,14.500000,17.500000,1761.400000,1770.420508,0.003086,1770.423594,2023-04-30 17:30:00.000000000,-0.053154,1770.370440,5.000000,-0.009576,1770.380016,9191.701610,980805.611191,757.60318,-970856.306400,331.381792,-971187.688192
5,6,1922.22,1922.22,1922.22,15,13,0.008,-22.3767,0,0,-44.7603,0,0,1675,-22.3767,-0.390547,-44.7603,-0.781215,15.216667,18.216667,1922.220000,1932.089099,0.002469,1932.091568,2023-04-30 18:13:00.000000001,-0.062611,1932.028957,5.716667,-0.010949,1932.039906,9353.361499,980806.103346,516.89830,-970935.843547,226.095520,-971161.939067
6,7,2015.22,2015.22,2015.22,15,45,0.010,-22.4063,0,0,-44.7524,0,0,1250,-22.4063,-0.391064,-44.7524,-0.781077,15.750000,18.750000,2015.220000,2025.582492,0.003086,2025.585578,2023-04-30 18:45:00.000000000,-0.063381,2025.522198,6.250000,-0.011970,2025.534168,9446.855761,980809.937020,385.74500,-970977.336258,168.728000,-971146.064258
7,8,2194.95,2194.95,2194.95,16,30,0.009,-22.4967,0,0,-44.6783,0,0,480,-22.4967,-0.392641,-44.6783,-0.779783,16.500000,19.500000,2194.950000,2206.301275,0.002777,2206.304053,2023-04-30 19:30:00.000000000,-0.055125,2206.248928,7.000000,-0.013407,2206.262335,9627.583928,980821.645707,148.12608,-971045.935699,64.791552,-971110.727251
8,1,2381.77,2381.77,2381.77,19,23,0.010,-22.8956,0,0,-43.2233,0,0,25,-22.8956,-0.399604,-43.2233,-0.754389,19.383333,22.383333,2381.770000,2394.163636,0.003086,2394.166722,2023-04-30 22:22:59.999999998,0.042756,2394.209477,9.883333,-0.018929,2394.228406,9815.550000,980873.318477,7.71490,-971050.053577,3.374560,-971053.428137
